<a href="https://colab.research.google.com/github/JonathanCristovao/LSTM_GRU_RNN_human-activity-recognition/blob/main/LSTM_GRU_RNN_data_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook was run on google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Library

In [ ]:
!pip install keras_self_attention

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from scipy import stats
from sklearn import metrics
from sklearn.metrics import classification_report
import glob
import csv
import os
%matplotlib inline
from keras_self_attention import SeqSelfAttention
import warnings
warnings.filterwarnings('ignore')

## The dataset used in this notebook can be downloaded here:
[Click here to download the Dataset](https://www.kaggle.com/datasets/jonathansilva2020/dataset-for-human-activity-recognition-har)



In [ ]:
interesting_dir = "/content/drive/MyDrive/dataset_aug_HAR"
interesting_labels = ['seated','stand','walk','iw_uphill']
save_filename = "activity"

## Concate and label the raw data  

In [ ]:
def data_concate_label(path,label):
    all_files1 = glob.glob(path + "/*.csv")
    
    li = []

    for filename in all_files1:
        df = pd.read_csv(filename, index_col=None, header=0)
        
        if 'label' in df.columns:
            pass
        else:df['label'] = str(label)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    if len(df.columns) == 7:#8 :
        new_header =['time','ID_S','Q_I','Q_J','Q_K','Q_R','label']
        #new_header =['date','time','ID_S','Q_I','Q_J','Q_K','Q_R','label']
    
    else :
       
        #new_header =['date','time','ID_S','Q_I','Q_J','Q_K','Q_R','label']
        new_header =['time','ID_S','Q_I','Q_J','Q_K','Q_R','label']

    frame = frame[:160000]#frame[:16000]
    
    print(frame.shape)
    print(len(df.columns))
    frame.to_csv("/content/drive/MyDrive/dataset_aug_HAR/activity_"+str(label)+".csv",index = False,header=new_header)
    

In [ ]:
path1 = "/content/drive/MyDrive/dataset_aug_HAR/seated"
label1 = "seated"
path2 = "/content/drive/MyDrive/dataset_aug_HAR/stand"
label2 = "stand"
path3 = "/content/drive/MyDrive/dataset_aug_HAR/walk"
label3 = "walk"
path4 = "/content/drive/MyDrive/dataset_aug_HAR/wuphill"
label4 = "iw_uphill"
data_concate_label(path1,label1)
data_concate_label(path2,label2)
data_concate_label(path3,label3)
data_concate_label(path4,label4)

## Concate all data into one .csv file

In [ ]:
import copy

df = pd.DataFrame()
for i in range(len(interesting_labels)):
    df_tmp = pd.read_csv(interesting_dir+"/"+save_filename+"_"+interesting_labels[i]+'.csv',header = 0)
    df = pd.concat([df, df_tmp])
print(len(df.columns))

drop_list = ['time']#['date','time']

df = df.drop(drop_list, axis=1)
df = df.dropna()

all_data_filename = "total.csv"
df.to_csv(interesting_dir+all_data_filename)

df.head()

## Feature correlation 

In [ ]:
df_cor = copy.deepcopy(df) 

features = df_cor.columns[0:-1]
df_cor["label"] = pd.factorize(df_cor.label)[0]
traindf = pd.DataFrame(df_cor,columns=features)

feature_name = list(traindf.columns)
print(feature_name)
y = df_cor['label']
X = traindf.copy()
X.isna().sum()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) 
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
#get correlations of each features in dataset
#corrmat = df_cor.corr()
#top_corr_features = corrmat.index
#plt.figure(figsize=(20,20))
#plot heat map
#g=sns.heatmap(df_cor[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
num_feats= len(feature_name)
X.columns.tolist()

In [ ]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
        
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
   
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]

    return cor_support, cor_feature, cor_list

cor_support, cor_feature, cor_list = cor_selector(X, y,num_feats)
cor_val=[abs(x) for x in cor_list]
cor_result = dict(zip(cor_feature, cor_val))
myDict = {key:val for key, val in cor_result.items() if val > 0}
{k: v for k, v in sorted(myDict.items(),reverse=True, key=lambda item: item[1])}

In [ ]:
activities = df['label'].value_counts().index
activities

## Split training and testing data

In [ ]:
def train_test_split(label, ratio):
    split_point = int(len(df[df.label == label]) * ratio)
    return (df[df.label == label].iloc[:split_point, :], df[df.label == label].iloc[split_point:, :])

split_ratio = 0.8
train_data = pd.DataFrame([])
test_data = pd.DataFrame([])

for i in range(len(interesting_labels)):
    (train, test) = train_test_split(interesting_labels[i], split_ratio)
    train_data = pd.concat([train_data, train])
    test_data = pd.concat([test_data, test])

print("Number of train samples: ", len(train_data))
print("Number of test samples: ", len(test_data))


train_label = train_data['label'].to_frame()
test_label = test_data['label'].to_frame()

## Data Segmentation (Sliding Window)

In [ ]:
N_TIME_STEPS = 5 #sliding window length # 10 #120
STEP = 2 # Sliding window step size 20
N_FEATURES = 5

def generate_sequence(x, y, n_time_steps, step):
    
    segments = []
    labels = []
    for i in range(0, len(x) - n_time_steps, step):

        ID_S = x['ID_S'].values[i: i + n_time_steps]
        Q_I = x['Q_I'].values[i: i + n_time_steps]
        Q_J = x['Q_J'].values[i: i + n_time_steps]
        Q_K = x['Q_K'].values[i: i + n_time_steps]
        Q_R = x['Q_R'].values[i: i + n_time_steps]
     
        label = stats.mode(y['label'][i: i + n_time_steps])[0][0]
        segments.append([ID_S, Q_I, Q_J, Q_K, Q_R])
        labels.append(label)
        
    return segments, labels

train_X, train_y = generate_sequence(train_data, train_label, N_TIME_STEPS, STEP)
test_X, test_y = generate_sequence(test_data, test_label, N_TIME_STEPS, STEP)

In [ ]:
# reshape input segments and one-hot encode labels
def reshape_segments(x, y, n_time_steps, n_features):
    
    x_reshaped = np.asarray(x, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    y_reshaped = np.asarray(pd.get_dummies(y), dtype = np.float32)
    return x_reshaped, y_reshaped

X_train, y_train = reshape_segments(train_X, train_y, N_TIME_STEPS, N_FEATURES)
X_test, y_test = reshape_segments(test_X, test_y, N_TIME_STEPS, N_FEATURES)

In [ ]:
#y_test

## Import DL Library

In [ ]:
from keras.regularizers import l2
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.layers import Attention
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras import optimizers
from keras import backend as K
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau
N_CLASSES = 4
L2 = 0.00001

## LSTM Model 

In [ ]:
# LSTM model
from keras.backend import clear_session
filepath = 'har_model_lstm_2021.h5'

clear_session()
model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=True,kernel_initializer='orthogonal',
                             kernel_regularizer=l2(L2), recurrent_regularizer=l2(L2),
                             bias_regularizer=l2(L2),name="LSTM_1"),
                             input_shape=(N_TIME_STEPS, N_FEATURES)))

model.add(Flatten(name='Flatten'))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_3"))
model.add(Dropout(0.2))
model.add(Dense(N_CLASSES, activation='softmax', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_4"))


model.summary()

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)
stop = EarlyStopping(monitor='val_loss', patience =5,
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)
reduce =  ReduceLROnPlateau(monitor='val_loss',
                    factor=0.5,
                    patience=2,
                    verbose=1)
callbacks = [checkpoint,stop]

model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

BATCH_SIZE =64
N_EPOCHS = 300

lstm = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE, shuffle=True,epochs=N_EPOCHS,callbacks=callbacks,
          validation_data=(X_test, y_test))

In [ ]:
# use LSTM model to predict

y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=y_pred_labels)
print(y_true_labels)
LABELS = ['seated', 'stand','walk','iwuphill']

plt.figure(figsize=(8, 6))
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix - LSTM")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
predIdxs = model.predict(X_test, batch_size=BATCH_SIZE)
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
## show a classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=['seated', 'stand','walk','iwuphill']))


In [ ]:
plt.plot(lstm.history['accuracy'])
plt.plot(lstm.history['val_accuracy'])
plt.title('model accuracy - LSTM')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim(0,1)
plt.legend(['train', 'val'], loc='best')
plt.show()

plt.plot(lstm.history['loss'])
plt.plot(lstm.history['val_loss'])
plt.title('model loss - LSTM')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
plt.show()

## Save the model into .pb file

In [ ]:
# Save LSTM model

#output_node_name = "Dense_2/Softmax"
#MODEL_LSTM = str(N_TIME_STEPS)+"_"+str(STEP)+"_"+str(N_FEATURES)+"_"+str(N_CLASSES)+"_nomagnew"

#tf.train.write_graph(K.get_session().graph_def, './models/LSTM/', \     MODEL_LSTM + '_graph.pbtxt')
#saver = tf.train.Saver()
#saver.save(K.get_session(), './models/LSTM/' + MODEL_LSTM + '.chkp')

#freeze_graph.freeze_graph('./models/LSTM/' +MODEL_LSTM+ '_graph.pbtxt', None, \
#   False, './models/LSTM/' + MODEL_LSTM+ '.chkp', output_node_name, \
#   "save/restore_all", "save/Const:0", \
#    './models/LSTM/' +MODEL_LSTM + '.pb', True, "")

# GRU Model

In [ ]:
from keras.layers import GRU

In [ ]:
# GRU model
from keras.backend import clear_session
filepath = 'har_model_gru.h5'

clear_session()
model = Sequential()
model.add(Bidirectional(GRU(200, return_sequences=True,kernel_initializer='orthogonal',
                             kernel_regularizer=l2(L2), recurrent_regularizer=l2(L2),
                             bias_regularizer=l2(L2),name="LSTM_1"),
                             input_shape=(N_TIME_STEPS, N_FEATURES)))

model.add(Flatten(name='Flatten'))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_3"))
model.add(Dropout(0.2))
model.add(Dense(N_CLASSES, activation='softmax', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_4"))


model.summary()

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)
stop = EarlyStopping(monitor='val_loss', patience =5,
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)
reduce =  ReduceLROnPlateau(monitor='val_loss',
                    factor=0.5,
                    patience=2,
                    verbose=1)
callbacks = [checkpoint,stop]

model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

BATCH_SIZE =64
N_EPOCHS = 300

lstm = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE, shuffle=True,epochs=N_EPOCHS,callbacks=callbacks,
          validation_data=(X_test, y_test))

In [ ]:
# GRU model to predict

y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=y_pred_labels)
print(y_true_labels)
LABELS = ['seated', 'stand','walk','iwuphill']

plt.figure(figsize=(8, 6))
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix- GRU")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();



In [ ]:
predIdxs = model.predict(X_test, batch_size=BATCH_SIZE)
predIdxs = np.argmax(predIdxs, axis=1)

# show a classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=['seated', 'stand','walk','iwuphill']))




In [ ]:
plt.plot(lstm.history['accuracy'])
plt.plot(lstm.history['val_accuracy'])
plt.title('model accuracy - GRU')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim(0,1)
plt.legend(['train', 'val'], loc='best')
plt.show()

plt.plot(lstm.history['loss'])
plt.plot(lstm.history['val_loss'])
plt.title('model loss - GRU')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
plt.show()



RNN Model

In [ ]:
from keras.layers import SimpleRNN

In [ ]:
# Simple RNN model
from keras.backend import clear_session
filepath = 'har_model_rnn.h5'

clear_session()
model = Sequential()

model.add(SimpleRNN(200,input_shape=(N_TIME_STEPS, N_FEATURES), return_sequences=True,kernel_initializer='orthogonal',
                             kernel_regularizer=l2(L2), recurrent_regularizer=l2(L2),
                             bias_regularizer=l2(L2),))


model.add(Flatten(name='Flatten'))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_3"))
model.add(Dropout(0.2))
model.add(Dense(N_CLASSES, activation='softmax', kernel_regularizer=l2(L2), bias_regularizer=l2(L2), name="Dense_4"))


model.summary()

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)
stop = EarlyStopping(monitor='val_loss', patience =5,
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)
reduce =  ReduceLROnPlateau(monitor='val_loss',
                    factor=0.5,
                    patience=2,
                    verbose=1)
callbacks = [checkpoint,stop]

model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

BATCH_SIZE =64
N_EPOCHS = 300

lstm = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE, shuffle=True,epochs=N_EPOCHS,callbacks=callbacks,
          validation_data=(X_test, y_test))

In [ ]:
# RNN model to predict

y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=y_pred_labels)
print(y_true_labels)
LABELS = ['seated', 'stand','walk','iwuphill']

plt.figure(figsize=(8, 6))
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix - SimpleRNN")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();


In [ ]:
predIdxs = model.predict(X_test, batch_size=BATCH_SIZE)
predIdxs = np.argmax(predIdxs, axis=1)

# show a classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=['seated', 'stand','walk','iwuphill']))

In [ ]:
plt.plot(lstm.history['accuracy'])
plt.plot(lstm.history['val_accuracy'])
plt.title('model accuracy - SimpleRNN')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim(0,1)
plt.legend(['train', 'val'], loc='best')
plt.show()

plt.plot(lstm.history['loss'])
plt.plot(lstm.history['val_loss'])
plt.title('model loss - SimpleRNN')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
plt.show()

